# Global Optimisers

- show basic stuff: 
  - simulated annealing
  - genetic algorithm

- scipy implementations: specification of constraints not possible
- need to find good penalty functions

In [36]:
import numpy as np
from scipy.optimize import shgo
#from scipy.optimize import Bounds
#from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
#from scipy.optimize import SR1

from beam import Cantilever

In [48]:
L = 1000
dstab = None
loads = [16000, -30000]
matname = 'TI64'
exr = -300 # €/kg, exchange rate

# x: [h, tw, blf, tlf, buf, tuf]
bounds = list(zip([50, 1.5, 6, 1, 6, 1], [300, 12, 100, 12, 100, 12]))
print(bounds)
x0 = [180, 3.2, 50, 6, 50, 6]

rfkeys = [
    'rf_t_uf', 
    'rf_t_lf',
    'rf_c_uf',
    'rf_c_lf',
    'rf_lb_uf',
    'rf_lb_lf',
    'rf_s_web',
    'rf_wb',
    'rf_lat',
]

def mass_penalty(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    result = b.analyse(loads)
    target = result['mass']
    rfmin = min(result[key] for key in rfkeys)
    if rfmin < 1:
        return target / rfmin
    else:
        return target


def cost_penalty(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    result = b.analyse(loads)
    target = result['cost']
    rfmin = min(result[key] for key in rfkeys)
    if rfmin < 1:
        return target / rfmin
    else:
        return target

def combined(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    return b.cost() - exr*b.mass()



# TODO: formulate linear constraint
# bf >= tw
#linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

# all rf >= 1
#nonlinear_constraint = NonlinearConstraint(cons_f, 1, np.inf) 

[(50, 300), (1.5, 12), (6, 100), (1, 12), (6, 100), (1, 12)]


In [52]:
res_m = shgo(mass_penalty, bounds, 
             sampling_method='sobol')

In [53]:
res_m

     fun: 3.318779278192845
    funl: array([3.31877928])
 message: 'Optimization terminated successfully.'
    nfev: 1119
     nit: 2
   nlfev: 919
   nlhev: 0
   nljev: 100
 success: True
       x: array([150.34102212,   2.59072643,  20.87845765,   9.69872951,
        36.57129529,   5.00897881])
      xl: array([[150.34102212,   2.59072643,  20.87845765,   9.69872951,
         36.57129529,   5.00897881]])

In [56]:
bm = Cantilever(matname, L, *res_m.x)
print(bm.mass(), bm.cost())
bm.analyse(loads)

3.3168067492521334 3416.601369731434


{'wmax': 36.20135683483408,
 'rf_t_uf': 1.0003198670334261,
 'rf_t_lf': 1.8826793349716902,
 'rf_c_uf': 1.7413353069623825,
 'rf_c_lf': 0.999406007956334,
 'rf_lb_uf': 7.312291495676757,
 'rf_lb_lf': 50.67121859733051,
 'rf_s_web': 4.06049291843876,
 'rf_wb': 0.9994056462405703,
 'rf_lat': 1.3998822656173837,
 'Fmax': 30000,
 'mass': 3.3168067492521334,
 'cost': 3416.601369731434,
 'area': 737.068166500474,
 'L': 1000,
 'h': 150.3410221177743,
 'tw': 2.590726425469373,
 'blf': 20.87845765070972,
 'tlf': 9.698729507122035,
 'buf': 36.57129528618308,
 'tuf': 5.008978805299209,
 'matname': 'TI64'}